# Cointab - Data Analyst
* #### Name - Jayant Sengupta
* #### Email - jayant.can@gmail.com
* #### Mob - +91 8435502241

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
from IPython.display import display
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
# importing X datasets
orders = pd.read_excel(r"G:\Company Assessments\Cointab\Data\Company X - Order Report.xlsx")
SKUs = pd.read_excel(r"G:\Company Assessments\Cointab\Data\Company X - SKU Master.xlsx")
zones = pd.read_excel(r"G:\Company Assessments\Cointab\Data\Company X - Pincode Zones.xlsx")

# importing Courier Company Datasets
invoice = pd.read_excel(r"G:\Company Assessments\Cointab\Data\Courier Company - Invoice.xlsx")
rates = pd.read_excel(r"G:\Company Assessments\Cointab\Data\Courier Company - Rates.xlsx")

#### Company X

In [25]:
display(orders)
display(SKUs)
display(zones)

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1
1,2001827036,8904223819093,1
2,2001827036,8904223819109,1
3,2001827036,8904223818430,1
4,2001827036,8904223819277,1
...,...,...,...
395,2001806229,8904223818942,1
396,2001806229,8904223818850,1
397,2001806226,8904223818850,2
398,2001806210,8904223816214,1


,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
114,121003,324008,b
115,121003,302020,b
119,121003,325207,b
120,121003,303702,b


#### Processing the data

In [6]:
# checking duplicate values
print("Orders duplicate values :-")
display(orders[orders.duplicated()])

print("\nSKUs duplicate values :-")
display(SKUs[SKUs.duplicated()])

print("\nZones duplicate values :-")
display(zones[zones.duplicated()])

Orders duplicate values :-


,ExternOrderNo,SKU,Order Qty
110,2001811363,8904223815859,1
317,2001806885,8904223819499,2



SKUs duplicate values :-


,SKU,Weight (g)
56,GIFTBOX202002,500



Zones duplicate values :-


,Warehouse Pincode,Customer Pincode,Zone
16,121003,140301,b
50,121003,248001,b
72,121003,302002,b
75,121003,302017,b
77,121003,313001,b
78,121003,313001,b
82,121003,313001,b
90,121003,302017,b
101,121003,335001,b
106,121003,302031,b


In [7]:
# removing duplicates
orders.drop_duplicates(inplace=True)
SKUs.drop_duplicates(inplace=True)
zones.drop_duplicates(inplace=True)

In [8]:
# merging orders and SKUs
orders_sku = pd.merge(orders, SKUs, how='inner', on='SKU')

# converting weight to Kg
orders_sku["Weight (g)"] = orders_sku['Weight (g)']/1000
orders_sku.rename(columns={'Weight (g)':'Weight(kg)'}, inplace = True)

display(orders_sku)

,ExternOrderNo,SKU,Order Qty,Weight(kg)
0,2001827036,8904223818706,1,0.127
1,2001821995,8904223818706,1,0.127
2,2001819252,8904223818706,1,0.127
3,2001816996,8904223818706,1,0.127
4,2001814580,8904223818706,1,0.127
...,...,...,...,...
393,2001806616,8904223819123,1,0.250
394,2001806567,8904223815804,1,0.160
395,2001806567,8904223818577,1,0.150
396,2001806408,8904223819437,2,0.552


In [9]:
# multiplying weight and qty to get total weight
orders_sku['Total Weight as per X(kg)'] = orders_sku['Weight(kg)'] * orders_sku['Order Qty']

# creating df by dropping some columns
df = orders_sku.drop(['SKU', 'Order Qty', 'Weight(kg)'], axis = 1)

# using groupby() to get total weight per order id
df = df.groupby('ExternOrderNo')['Total Weight as per X(kg)'].sum().reset_index()

df.head()

,ExternOrderNo,Total Weight as per X(kg)
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245


In [10]:
# create a funtion to assign slabs
def assigned_slab(value):
    
    if value <= 0:
        return value
    
    else:
        assigned_slab = math.ceil(value * 2) / 2
        return assigned_slab

In [11]:
# utilsing function to assign slabs
df['Weight slab as per X(kg)'] = df['Total Weight as per X(kg)'].apply(assigned_slab)
df.head()

,ExternOrderNo,Total Weight as per X(kg),Weight slab as per X(kg)
0,2001806210,0.220,0.5
1,2001806226,0.480,0.5
2,2001806229,0.500,0.5
3,2001806232,1.302,1.5
4,2001806233,0.245,0.5


In [12]:
# merging 'df' with 'invoice' on orderID
df_invoice = pd.merge( invoice, df, how='inner', left_on='Order ID', right_on='ExternOrderNo' )
df_invoice.drop(columns = 'ExternOrderNo', inplace=True)

df_invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Total Weight as per X(kg),Weight slab as per X(kg)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5


In [13]:
# merging 'df_invoice' with 'zones' on common columns to get 'Warehouse and Destination Pincodes'
merged = pd.merge(df_invoice, zones, how='left', on=['Warehouse Pincode', 'Customer Pincode'])
merged.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Total Weight as per X(kg),Weight slab as per X(kg),Zone_y
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5,d
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0,d
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5,d
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0,b
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5,d


In [14]:
# renaming columns
merged.rename(columns={'AWB Code':'AWB Number',
                       'Zone_x':'Delivery Zone charged by Courier Company',
                       'Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)',
                       'Zone_y':'Delivery Zone as per X',
                       'Charged Weight':'Total weight as per Courier Company (KG)'}, inplace=True)

# applying slab function to get slabs charged by the courier company 
merged['Weight slab as per Courier Company(kg)'] = merged['Total weight as per Courier Company (KG)'].apply(assigned_slab)

# rearranging columns
report = merged[['Order ID', 'AWB Number', 'Total Weight as per X(kg)', 'Weight slab as per X(kg)', 
                'Total weight as per Courier Company (KG)', 'Weight slab as per Courier Company(kg)', 
                'Delivery Zone as per X', 'Delivery Zone charged by Courier Company', 'Charges Billed by Courier Company (Rs.)',
                'Warehouse Pincode', 'Customer Pincode', 'Type of Shipment']]

merged.head()

,AWB Number,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total Weight as per X(kg),Weight slab as per X(kg),Delivery Zone as per X,Weight slab as per Courier Company(kg)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5,d,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0,d,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5,d,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0,b,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5,d,0.5


In [15]:
# creating a bool column to indicate rows where zone have been wrongly charged
report['Zone wrongly assigned'] = [True 
                                   if x != y
                                   else False 
                                   for x, y in 
                                   zip(report['Delivery Zone as per X'], report['Delivery Zone charged by Courier Company'])
                                  ]

# creating a bool column to indicate where additional charges are applicable
report['Additional applicable'] = [True 
                                   if x > 0.5
                                   else False 
                                   for x in report['Weight slab as per X(kg)']]

report.head()

,Order ID,AWB Number,Total Weight as per X(kg),Weight slab as per X(kg),Total weight as per Courier Company (KG),Weight slab as per Courier Company(kg),Delivery Zone as per X,Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Warehouse Pincode,Customer Pincode,Type of Shipment,Zone wrongly assigned,Additional applicable
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,121003,507101,Forward charges,False,True
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,121003,486886,Forward charges,False,True
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,121003,532484,Forward charges,False,True
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.3,121003,143001,Forward charges,False,True
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,121003,515591,Forward charges,False,False


In [64]:
rates

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [17]:
# extracting zones
columns = rates.columns
tempSet = set()

for i in columns:
    tempSet.add(i[4:5])
    
tempList = list(tempSet)
tempList.sort()
tempList

['a', 'b', 'c', 'd', 'e']

In [18]:
rateCard = pd.DataFrame({'Zones':tempList})
rateCard

,Zones
0,a
1,b
2,c
3,d
4,e


In [19]:
# creating a function for extracting rates corresponding to each 'zone' and 'ship'
def fwd_fixed_charges(zone):
    fwd_fixed = rates[f'fwd_{zone}_fixed']
    return fwd_fixed
    
def fwd_additional_charges(zone):
    fwd_additional = rates[f'fwd_{zone}_additional']
    return fwd_additional

def rto_fixed_charges(zone):
    rto_fixed = rates[f'rto_{zone}_fixed']
    return rto_fixed
    
def rto_additional_charges(zone):    
    rto_additional = rates[f'rto_{zone}_additional']
    return rto_additional

In [20]:
# applying function 
rateCard['fwd_fixed']      = rateCard['Zones'].apply(fwd_fixed_charges)
rateCard['fwd_additional'] = rateCard['Zones'].apply(fwd_additional_charges)
rateCard['rto_fixed']      = rateCard['Zones'].apply(rto_fixed_charges)
rateCard['rto_additional'] = rateCard['Zones'].apply(rto_additional_charges)

In [21]:
rateCard

,Zones,fwd_fixed,fwd_additional,rto_fixed,rto_additional
0,a,29.5,23.6,13.6,23.6
1,b,33.0,28.3,20.5,28.3
2,c,40.1,38.9,31.9,38.9
3,d,45.4,44.8,41.3,44.8
4,e,56.6,55.5,50.7,55.5


In [22]:
# creating a function which will calculate total charges based on Zone, Slab and Shipment type
def calculate_total_charges(report_df, ratecard_df):
    
    # merging 'report' and 'ratecard' on common columns
    merged_df = pd.merge(report, rateCard, left_on= 'Delivery Zone as per X', right_on="Zones")
    
    # creating a new column to hold the total charges
    merged_df["Expected Charge as per X (Rs.)"] = 0
    
    # iterating through each row in the merged dataframe
    for index, row in merged_df.iterrows():
        
        # determining which charges to apply based on the shipment type
        if row["Type of Shipment"] == "Forward charges":
            charges = row["fwd_fixed"] + (row["fwd_additional"] * ((row["Weight slab as per X(kg)"] - 0.5) // 0.5))
            
        elif row["Type of Shipment"] == "Forward and RTO charges":
             charges = row["fwd_fixed"] + row["rto_fixed"] + (row["fwd_additional"] + 
                       row["rto_additional"]) * ((row["Weight slab as per X(kg)"] - 0.5) // 0.5)
            
        else:
            charges = 0
        
        # adding the charges to the 'Expected Charge' column for this row
        merged_df.loc[index, "Expected Charge as per X (Rs.)"] = charges
    
    return merged_df

# applying the function
final = calculate_total_charges(report, rateCard)

#### Explanation

1. In this loop, we're iterating through each row of the merged dataframe using the iterrows() method provided by pandas. This method returns an iterator that yields pairs of index-value pairs, where the index is the row index and the value is a pandas series containing the data in the row.

2. On each iteration of the loop, we're assigning the index and value to the variables index and row, respectively. Then we're determining which charges to apply based on the value of the "shipment_type" column in the current row.

* If the shipment type is "Fwd charges", we calculate the charges using the formula fwd_fixed_charge + (fwd_additional_charge * ((weight_slab - 0.5) // 0.5)). This formula applies the fixed charge for the weight slab, and then adds additional charges for each 0.5 KG above the initial 0.5 KG, using integer division to calculate the number of additional 0.5 KGs.

* If the shipment type is "Forward and rto charges", we calculate the charges using the formula fwd_fixed_charge + rto_fixed_charge + (fwd_additional_charge + rto_additional_charge) * ((weight_slab - 0.5) // 0.5). This formula is similar to the first one, but includes both the "fwd" and "rto" charges, and adds them together before calculating the additional charges.

* If the shipment type is anything else, we set the charges to 0.

3. Finally, we add the calculated charges to the "total_charges" column for the current row using the loc method provided by pandas. This method allows us to set the value of a specific cell in a dataframe, using the row index and column name as arguments. In this case, we're setting the value of the "total_charges" column for the current row to the calculated charges.

In [23]:
# arranging and dropping few columns for the desired output
final.drop(columns=['Warehouse Pincode', 'Customer Pincode', 'Type of Shipment',
                    'Zone wrongly assigned', 'Additional applicable', 'Zones', 'fwd_fixed', 
                    'fwd_additional', 'rto_fixed', 'rto_additional'], inplace=True)

# rearraning columns
final = final.iloc[:, [0,1,2,3,4,5,6,7,9,8]]

# creating final column to check the difference between estimated and actual 
final['Difference Between Expected Charges and Billed Charges (Rs.)'] = round(
                                                                        final['Charges Billed by Courier Company (Rs.)'] - 
                                                                        final['Expected Charge as per X (Rs.)'], 2
                                                                        )

#### Output 1

In [24]:
final

,Order ID,AWB Number,Total Weight as per X(kg),Weight slab as per X(kg),Total weight as per Courier Company (KG),Weight slab as per Courier Company(kg),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
4,2001806686,1091117229555,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,0.721,1.0,0.80,1.0,e,b,112.1,61.3,-50.8
120,2001819252,1091120352712,0.270,0.5,0.30,0.5,e,b,56.6,33.0,-23.6
121,2001827036,1091122418320,1.676,2.0,1.60,2.0,e,b,223.1,117.9,-105.2
122,2001808585,1091117436652,0.500,0.5,0.72,1.0,e,b,56.6,61.3,4.7


#### Output 2

In [62]:
# grouping OUTPUT 1 by the "difference" column
final2 = final.groupby("Difference Between Expected Charges and Billed Charges (Rs.)").size().reset_index(name="Count")

# calculating the total amount for overcharged and undercharged orders
overcharged = final[final["Difference Between Expected Charges and Billed Charges (Rs.)"] > 0]["Difference Between Expected Charges and Billed Charges (Rs.)"].sum()
undercharged = final[final["Difference Between Expected Charges and Billed Charges (Rs.)"] < 0]["Difference Between Expected Charges and Billed Charges (Rs.)"].sum() * -1

# calculating the total amount for correctly charged orders
correctly_charged = final[final["Difference Between Expected Charges and Billed Charges (Rs.)"] == 0]["Charges Billed by Courier Company (Rs.)"].sum()

# creating a new dataframe with two columns, "Count" and "Total Amount"
summary = pd.DataFrame({
    "Count": [final2[final2["Difference Between Expected Charges and Billed Charges (Rs.)"] == 0]["Count"].values[0], 
              final2[final2["Difference Between Expected Charges and Billed Charges (Rs.)"] > 0]["Count"].sum(), 
              final2[final2["Difference Between Expected Charges and Billed Charges (Rs.)"] < 0]["Count"].sum()],
    "Total Amount": [correctly_charged, overcharged, undercharged]
},
columns=["Count", "Total Amount"],
index=["Total orders where X has been correctly charged", 
       "Total Orders where X has been overcharged",
       "Total Orders where X has been undercharged"])

In [63]:
summary

,Count,Total Amount
Total orders where X has been correctly charged,21,1765.6
Total Orders where X has been overcharged,80,4483.2
Total Orders where X has been undercharged,23,575.1


In [65]:
# exporitng Output 1 and 2
final.to_csv('Output 1.csv')
summary.to_csv('Output 2.csv')